<a href="https://colab.research.google.com/github/RiccardoElisei/Tesi-IDS/blob/main/Tesi%20IDS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import glob

In [ ]:
path = "drive/MyDrive/MachineLearningCVE/"
files = [file for file in glob.glob(path + "**/*.csv", recursive=True)]

In [ ]:
[print(f) for f in files]

In [ ]:
dataset = [pd.read_csv(f) for f in files]

In [ ]:
for d in dataset:
    print(d.shape)

In [ ]:
dataset = pd.concat([d for d in dataset]).drop_duplicates(keep=False)
dataset.reset_index(drop=True, inplace = True)

In [ ]:
dataset.shape

In [ ]:
dataset.info()

In [ ]:
dataset.describe()

In [ ]:
dataset.head()

In [ ]:
print(dataset[' Label'].unique())
len(dataset[' Label'].unique())

In [ ]:
label_names = dataset[' Label'].unique()

import re

label_names = [re.sub("[^a-zA-Z ]+", "", l) for l in label_names]
label_names = [re.sub("[\s\s]", '_', l) for l in label_names]
label_names = [lab.replace("__", "_") for lab in label_names]

label_names, len(label_names)

In [ ]:
labels = dataset[' Label'].unique()

for i in range(0,len(label_names)):
    dataset[' Label'] = dataset[' Label'].replace({labels[i] : label_names[i]})

dataset[' Label'].unique()

In [ ]:
dataset.isnull().values.any()

In [ ]:
[col for col in dataset if dataset[col].isnull().values.any()]

In [ ]:
dataset['Flow Bytes/s'].isnull().sum()

In [ ]:
before = dataset.shape
dataset.dropna(inplace=True)
after = dataset.shape
before[0] - after[0]

In [ ]:
dataset.isnull().any().any()

In [ ]:
import numpy as np

In [ ]:
labl = dataset[' Label']
dataset = dataset.loc[:, dataset.columns != ' Label'].astype('float64')

In [ ]:
np.all(np.isfinite(dataset))

In [ ]:
nonfinite = [col for col in dataset if not np.all(np.isfinite(dataset[col]))]
nonfinite

In [ ]:
finite = np.isfinite(dataset['Flow Bytes/s']).sum()

dataset.shape[0] - finite

In [ ]:
finite = np.isfinite(dataset[' Flow Packets/s']).sum()

dataset.shape[0] - finite

In [ ]:
dataset = dataset.replace([np.inf, -np.inf], np.nan)

In [ ]:
dataset = dataset.merge(labl, how='outer', left_index=True, right_index=True)

In [ ]:
dataset.dropna(inplace=True)

In [ ]:
dataset.head()

In [ ]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
np.random.seed(42)
rand_perm = np.random.permutation(dataset.shape[0])

In [ ]:
feature_cols = dataset.columns[:-1]

dataset_subset = dataset.iloc[rand_perm[:10000],:]
dataset_subset

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X = dataset_subset[feature_cols]
Y = dataset_subset[' Label']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Crea un classificatore basato su alberi decisionali (Random Forest)
clf = RandomForestClassifier()

# Addestra il classificatore sul set di addestramento
clf.fit(X_train, Y_train)

# Valuta le prestazioni del modello sul set di test
Y_pred = clf.predict(X_test)
accuracy_rf = accuracy_score(Y_test, Y_pred)

# Visualizza l'importanza delle caratteristiche
feature_importance = clf.feature_importances_

# Crea un DataFrame per organizzare le informazioni
feature_df = pd.DataFrame({'Feature': X.columns, 'Importance': feature_importance})

# Ordina le feature per importanza
feature_df = feature_df.sort_values(by='Importance', ascending=False)

weights_rf = {feature: weight for weight, feature in enumerate(feature_df['Feature'], start=1)}

# Visualizza il grafico delle feature importance con plt.bar
plt.figure(figsize=(15, 8))
plt.bar(feature_df['Feature'][:50], feature_df['Importance'][:50], color='skyblue')
plt.xlabel('Feature')
plt.ylabel('Importance')
plt.title('Feature RandomForsetClassifier')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

print(feature_df.head(50))  # Mostra le prime 50 feature più importanti

print(accuracy_rf)

In [ ]:
# Seleziona le prime 20 feature dal classificatore RandomForest
top_features_rf = feature_df['Feature'][:20].tolist()

# Aggiungi il label al set di feature selezionate
selected_features_rf = [' Label'] + top_features_rf

# Crea un nuovo dataset con solo le prime 20 feature e il label
new_dataset_rf = dataset_subset[selected_features_rf]

# Visualizza le prime righe del nuovo dataset
print(new_dataset_rf.head())

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix

# Separazione delle feature e del label
X_rf = new_dataset_rf.drop(' Label', axis=1)  # Rimuovi la colonna del label
y_rf = new_dataset_rf[' Label']

# Creazione dei set di allenamento e test
X_train_rf, X_test_rf, y_train_rf, y_test_rf = train_test_split(X_rf, y_rf, test_size=0.2, random_state=42)

# Definizione del modello RandomForest
model_rf = RandomForestClassifier()

# Definizione della griglia dei parametri da cercare
param_grid = {"max_depth": [3, None], "min_samples_split": [5, 10]}

# Ricerca dei parametri migliori con Halving Grid Search CV
search = HalvingGridSearchCV(model_rf, param_grid, random_state=42).fit(X_train_rf, y_train_rf)
best_rf_params = search.best_params_

# Visualizzazione dei parametri migliori
print("Migliori parametri per RandomForest:", best_rf_params)

# Utilizzo dei migliori parametri per allenare il modello
best_model_rf = RandomForestClassifier(**best_rf_params)
best_model_rf.fit(X_train_rf, y_train_rf)

# Valutazione del modello RandomForest con i migliori parametri
predictions_rf_best = best_model_rf.predict(X_test_rf)
accuracy_rf_best = accuracy_score(y_test_rf, predictions_rf_best)
confusion_matrix_rf_best = confusion_matrix(y_test_rf, predictions_rf_best)

# Visualizza i risultati con i migliori parametri
print("Accuratezza RandomForest RFC con i migliori parametri:", accuracy_rf_best)
print("Matrice di confusione RandomForest RFC con i migliori parametri:\n", confusion_matrix_rf_best)

In [ ]:
import matplotlib.pyplot as plt

# Valori della matrice di confusione
TN = 1691
TP = 0
FP = 0
FN = 309

# Etichette e colori
labels = ['True Negative', 'False Positive', 'False Negative', 'True Positive']
colors = ['green', 'red', 'orange', 'blue']

# Crea il grafico a barre
plt.bar(labels, [TN, FP, FN, TP], color=colors)
plt.xlabel('Classificazioni')
plt.ylabel('Numero di campioni')
plt.title('Matrice di Confusione')
plt.show()

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix

X_svm_rf = new_dataset_rf.drop(' Label', axis=1)  # Rimuovi la colonna del label
y_svm_rf = new_dataset_rf[' Label']

# Creazione dei set di allenamento e test
X_train_svm_rf, X_test_svm_rf, y_train_svm_rf, y_test_svm_rf = train_test_split(X_svm_rf, y_svm_rf, test_size=0.2, random_state=42)

# Definisci il modello
model_svm = SVC()

# Definisci la griglia degli iperparametri
param_grid = {'C': [0.1, 0.5, 1, 2, 5, 10], 'kernel': ['linear', 'rbf', 'sigmoid', 'poly'], 'tol': [1e-3, 1e-2]}

# Applica Halving Grid Search CV
search_svm = HalvingGridSearchCV(model_svm, param_grid, cv=4, factor=2, max_resources=110, min_resources = 10, aggressive_elimination=False, random_state = 42)
search_svm.fit(X_train_svm_rf, y_train_svm_rf)

# Ottieni i migliori parametri
best_svm_params = search_svm.best_params_

# Visualizzazione dei parametri migliori
print("Migliori parametri per SVM:", best_svm_params)

# Utilizzo dei migliori parametri per allenare il modello
best_model_svm_rf = SVC(**best_svm_params)
best_model_svm_rf.fit(X_train_svm_rf, y_train_svm_rf)

# Valutazione del modello RandomForest con i migliori parametri
predictions_svm_rf = best_model_svm_rf.predict(X_test_svm_rf)
accuracy_svm_rf = accuracy_score(y_test_svm_rf, predictions_svm_rf)
confusion_matrix_svm_rf = confusion_matrix(y_test_svm_rf, predictions_svm_rf)

# Visualizza i risultati con i migliori parametri
print("Accuratezza SVM RFC con i migliori parametri:", accuracy_svm_rf)
print("Matrice di confusione SVM RFC con i migliori parametri:\n", confusion_matrix_svm_rf)


In [ ]:
#Random Forest Test RFC

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

# Separazione delle feature e del label
X_rf = new_dataset_rf.drop(' Label', axis=1)  # Rimuovi la colonna del label
y_rf = new_dataset_rf[' Label']


# Creazione dei set di allenamento e test
X_train_rf, X_test_rf, y_train_rf, y_test_rf = train_test_split(X_rf, y_rf, test_size=0.2, random_state=42)

# Allenamento del modello RandomForest
model_rf = RandomForestClassifier()
model_rf.fit(X_train_rf, y_train_rf)

# Valutazione del modello RandomForest
predictions_rf = model_rf.predict(X_test_rf)
accuracy_rf = accuracy_score(y_test_rf, predictions_rf)
confusion_matrix_rf = confusion_matrix(y_test_rf, predictions_rf)

# Visualizza i risultati
print("Accuratezza RandomForest RFC:", accuracy_rf)
print("Matrice di confusione RandomForest RFC:\n", confusion_matrix_rf)


In [ ]:
#Random Forest Test RFC attacchi

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Separazione delle feature e del label
X_rf = new_dataset_rf.drop(' Label', axis=1)  # Rimuovi la colonna del label
y_rf = new_dataset_rf[' Label']


# Creazione dei set di allenamento e test
X_train_rf, X_test_rf, y_train_rf, y_test_rf = train_test_split(X_rf, y_rf, test_size=0.2, random_state=42)

# Allenamento del modello RandomForest
model_rf = RandomForestClassifier()
model_rf.fit(X_train_rf, y_train_rf)

# Valutazione del modello RandomForest
predictions_rf = model_rf.predict(X_test_rf)
accuracy_rf = accuracy_score(y_test_rf, predictions_rf)
confusion_matrix_rf = confusion_matrix(y_test_rf, predictions_rf)

# Visualizza i risultati
print("Accuratezza RandomForest RFC:", accuracy_rf)
print("Matrice di confusione RandomForest RFC:\n", confusion_matrix_rf)

class_report_rf = classification_report(y_test_rf, predictions_rf)
print("Classification Report - Random Forest RFC:\n", class_report_rf)

# Estrai le etichette delle classi dal modello
class_labels_rf = model_rf.classes_

# Per ogni classe, estrai FN e FP dalla matrice di confusione
for i, label in enumerate(class_labels_rf):
    fn = confusion_matrix_rf[i, :].sum() - confusion_matrix_rf[i, i]  # Somma della riga, escludendo la diagonale
    fp = confusion_matrix_rf[:, i].sum() - confusion_matrix_rf[i, i]  # Somma della colonna, escludendo la diagonale

    print(f"\nClasse: {label}")
    print(f"False Negative (FN): {fn}")
    print(f"False Positive (FP): {fp}")


In [ ]:
#SVM test RFC attacchi


from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Separazione delle feature e del label
X_svm_rf = new_dataset_rf.drop(' Label', axis=1)  # Rimuovi la colonna del label
y_svm_rf = new_dataset_rf[' Label']

# Creazione dei set di allenamento e test
X_train_svm_rf, X_test_svm_rf, y_train_svm_rf, y_test_svm_rf = train_test_split(X_svm_rf, y_svm_rf, test_size=0.2, random_state=42)

# Allenamento del modello SVM
model_svm = SVC()
model_svm.fit(X_train_svm_rf, y_train_svm_rf)

# Valutazione del modello SVM
predictions_svm_rf = model_svm.predict(X_test_svm_rf)
accuracy_svm_rf = accuracy_score(y_test_svm_rf, predictions_svm_rf)
confusion_matrix_svm_rf = confusion_matrix(y_test_svm_rf, predictions_svm_rf)

# Ottieni gli iperparametri utilizzati nel modello SVM
svm_params = model_svm.get_params()

# Visualizza gli iperparametri
print("Iperparametri utilizzati nel modello SVM:")
for param, value in svm_params.items():
    print(f"{param}: {value}")

# Visualizza i risultati
print("Accuratezza SVM RFC:", accuracy_svm_rf)
print("Matrice di Confusione SVM RFC:\n", confusion_matrix_svm_rf)

class_report_svm_rf = classification_report(y_test_svm_rf, predictions_svm_rf)
print("Classification Report - Random Forest RFC:\n", class_report_svm_rf)

# Estrai le etichette delle classi dal modello
class_labels_svm_rf = model_svm.classes_

# Per ogni classe, estrai FN e FP dalla matrice di confusione
for i, label in enumerate(class_labels_svm_rf):
    fn = confusion_matrix_svm_rf[i, :].sum() - confusion_matrix_svm_rf[i, i]  # Somma della riga, escludendo la diagonale
    fp = confusion_matrix_svm_rf[:, i].sum() - confusion_matrix_svm_rf[i, i]  # Somma della colonna, escludendo la diagonale

    print(f"\nClasse: {label}")
    print(f"False Negative (FN): {fn}")
    print(f"False Positive (FP): {fp}")

In [ ]:
#SVM Test RFC

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

# Separazione delle feature e del label
X_svm_rf = new_dataset_rf.drop(' Label', axis=1)  # Rimuovi la colonna del label
y_svm_rf = new_dataset_rf[' Label']

# Creazione dei set di allenamento e test
X_train_svm_rf, X_test_svm_rf, y_train_svm_rf, y_test_svm_rf = train_test_split(X_svm_rf, y_svm_rf, test_size=0.2, random_state=42)

# Allenamento del modello SVM
model_svm = SVC()
model_svm.fit(X_train_svm_rf, y_train_svm_rf)

# Valutazione del modello SVM
predictions_svm_rf = model_svm.predict(X_test_svm_rf)
accuracy_svm_rf = accuracy_score(y_test_svm_rf, predictions_svm_rf)
confusion_matrix_svm_rf = confusion_matrix(y_test_svm_rf, predictions_svm_rf)

# Ottieni gli iperparametri utilizzati nel modello SVM
svm_params = model_svm.get_params()

# Visualizza gli iperparametri
print("Iperparametri utilizzati nel modello SVM:")
for param, value in svm_params.items():
    print(f"{param}: {value}")

# Visualizza i risultati
print("Accuratezza SVM RFC:", accuracy_svm_rf)
print("Matrice di Confusione SVM RFC:\n", confusion_matrix_svm_rf)


In [ ]:
!pip install boruta
import pandas as pd
from boruta import BorutaPy
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

# separiamo le features (X) dal target (Y)
X = dataset_subset[feature_cols]
Y = dataset_subset[' Label']

# Inizializza il classificatore RandomForest
rf = RandomForestClassifier(n_jobs=-1, class_weight='balanced', max_depth=5)

# Inizializza Boruta
boruta_selector = BorutaPy(rf, n_estimators= 50 , verbose=0, max_iter = 100)

# Addestra Boruta sul dataset
boruta_selector.fit(X.values, Y.values)

# Ottieni le caratteristiche selezionate e i loro ranghi
selected_features = X.columns[boruta_selector.support_]
ranks = boruta_selector.ranking_

print("Dimensione di selected_features:", len(selected_features))
print("Dimensione di boruta_selector.ranking_:", len(ranks))
print("Selected Features:", selected_features[:50])
print("Ranks:", boruta_selector.ranking_[:50])

# Seleziona solo le colonne corrispondenti alle caratteristiche selezionate
X_selected = X[selected_features]

# Suddividi il dataset in set di addestramento e set di test
X_train, X_test, Y_train, Y_test = train_test_split(X_selected, Y, test_size=0.2, random_state=42)

# Inizializza un nuovo classificatore RandomForest
rf_selected_features = RandomForestClassifier(n_jobs=-1, class_weight='balanced', max_depth=5)

# Addestra il classificatore solo sulle caratteristiche selezionate
rf_selected_features.fit(X_train.values, Y_train.values)

# Effettua le previsioni sul set di test
Y_pred = rf_selected_features.predict(X_test.values)

# Calcola l'accuratezza
accuracy_boruta = accuracy_score(Y_test, Y_pred)

feature_importances = rf_selected_features.feature_importances_

feature_weights = {feature: importance for feature, importance in zip(selected_features, feature_importances)}

# Ordina le feature in base all'importanza
sorted_features = sorted(feature_weights, key=feature_weights.get, reverse=True)

# Assegna i pesi in base all'ordine di importanza
weights_boruta = {feature: weight for feature, weight in zip(sorted_features, range(1, len(sorted_features) + 1))}

# Crea un DataFrame con le caratteristiche e i loro ranghi
boruta_results = pd.DataFrame({'Feature': selected_features[:50], 'Rank': boruta_selector.ranking_[:50]})
boruta_results = boruta_results.sort_values(by='Rank', ascending=True)

# Visualizza i risultati con un grafico a barre
plt.figure(figsize=(12, 6))
plt.bar(range(len(boruta_results)), boruta_results['Rank'], tick_label=boruta_results['Feature'], color = 'skyblue')
plt.xticks(rotation=90)
plt.title('Boruta')
plt.xlabel('Caratteristiche')
plt.ylabel('Rango')
plt.show()

print(boruta_results)

print(accuracy_boruta)


In [ ]:
# Seleziona le prime 20 feature da Boruta
top_features_boruta = boruta_results['Feature'][-20:][::-1].tolist()

# Aggiungi la colonna del label al set di feature selezionate
selected_features_boruta = [' Label'] + top_features_boruta

# Crea un nuovo dataset con solo le prime 20 feature e il label
new_dataset_boruta = dataset_subset[selected_features_boruta]

# Visualizza le prime righe del nuovo dataset
print(new_dataset_boruta.head())


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix

X_boruta = new_dataset_boruta.drop(' Label', axis=1)  # Rimuovi la colonna del label
y_boruta = new_dataset_boruta[' Label']

# Creazione dei set di allenamento e test
X_train_boruta, X_test_boruta, y_train_boruta, y_test_boruta = train_test_split(X_boruta, y_boruta, test_size=0.2, random_state=42)

# Definizione del modello RandomForest
model_boruta = RandomForestClassifier()

# Definizione della griglia dei parametri da cercare
param_grid = {"max_depth": [3, None], "min_samples_split": [5, 10]}

# Ricerca dei parametri migliori con Halving Grid Search CV
search = HalvingGridSearchCV(model_boruta, param_grid, random_state=42).fit(X_train_boruta, y_train_boruta)
best_boruta_params = search.best_params_

# Visualizzazione dei parametri migliori
print("Migliori parametri per RandomForestClassifier:", best_boruta_params)

# Utilizzo dei migliori parametri per allenare il modello
best_model_boruta = RandomForestClassifier(**best_boruta_params)
best_model_boruta.fit(X_train_boruta, y_train_boruta)

# Valutazione del modello RandomForest con i migliori parametri
predictions_boruta_best = best_model_boruta.predict(X_test_boruta)
accuracy_boruta_best = accuracy_score(y_test_boruta, predictions_boruta_best)
confusion_matrix_boruta_best = confusion_matrix(y_test_boruta, predictions_boruta_best)

# Visualizza i risultati con i migliori parametri
print("Accuratezza RandomForest Boruta con i migliori parametri:", accuracy_boruta_best)
print("Matrice di confusione RandomForest Boruta con i migliori parametri:\n", confusion_matrix_boruta_best)




In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

# Separazione delle feature e del label
X_svm_b = new_dataset_boruta.drop(' Label', axis=1)  # Rimuovi la colonna del label
y_svm_b = new_dataset_boruta[' Label']

# Creazione dei set di allenamento e test
X_train_svm_b, X_test_svm_b, y_train_svm_b, y_test_svm_b = train_test_split(X_svm_b, y_svm_b, test_size=0.2, random_state=42)

# Definisci il modello
model_svm_b = SVC()

# Definisci la griglia degli iperparametri
param_grid = {'C': [0.1, 0.5, 1, 2, 5, 10], 'kernel': ['linear', 'rbf', 'sigmoid', 'poly'], 'tol': [1e-3, 1e-2]}

# Applica Halving Grid Search CV
search_svm = HalvingGridSearchCV(model_svm_b, param_grid, cv=5, factor=2, max_resources=110, min_resources=10, aggressive_elimination=False, random_state = 42)
search_svm.fit(X_train_svm_b, y_train_svm_b)

# Ottieni i migliori parametri
best_svm_params = search_svm.best_params_

# Visualizzazione dei parametri migliori
print("Migliori parametri per SVM:", best_svm_params)

# Utilizzo dei migliori parametri per allenare il modello
best_model_svm_b = SVC(**best_svm_params)
best_model_svm_b.fit(X_train_svm_b, y_train_svm_b)

# Valutazione del modello RandomForest con i migliori parametri
predictions_svm_b = best_model_svm_b.predict(X_test_svm_b)
accuracy_svm_b = accuracy_score(y_test_svm_b, predictions_svm_b)
confusion_matrix_svm_b = confusion_matrix(y_test_svm_b, predictions_svm_b)

# Visualizza i risultati con i migliori parametri
print("Accuratezza SVM Boruta con i migliori parametri:", accuracy_svm_b)
print("Matrice di confusione SVM Boruta con i migliori parametri:\n", confusion_matrix_svm_b)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

X_boruta = new_dataset_boruta.drop(' Label', axis=1)  # Rimuovi la colonna del label
y_boruta = new_dataset_boruta[' Label']

# Creazione dei set di allenamento e test
X_train_boruta, X_test_boruta, y_train_boruta, y_test_boruta = train_test_split(X_boruta, y_boruta, test_size=0.2, random_state=42)

# Allenamento del modello RandomForest
model_boruta = RandomForestClassifier()
model_boruta.fit(X_train_boruta, y_train_boruta)

# Valutazione del modello RandomForest
predictions_boruta = model_boruta.predict(X_test_boruta)
accuracy_boruta = accuracy_score(y_test_boruta, predictions_boruta)
confusion_matrix_boruta = confusion_matrix(y_test_boruta, predictions_boruta)

# Visualizza i risultati
print("Accuratezza RandomForest Boruta:", accuracy_boruta)
print("Matrice di confusione RandomForest Boruta:\n", confusion_matrix_boruta)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

X_boruta = new_dataset_boruta.drop(' Label', axis=1)  # Rimuovi la colonna del label
y_boruta = new_dataset_boruta[' Label']

# Creazione dei set di allenamento e test
X_train_boruta, X_test_boruta, y_train_boruta, y_test_boruta = train_test_split(X_boruta, y_boruta, test_size=0.2, random_state=42)

# Allenamento del modello RandomForest
model_boruta = RandomForestClassifier()
model_boruta.fit(X_train_boruta, y_train_boruta)

# Valutazione del modello RandomForest
predictions_boruta = model_boruta.predict(X_test_boruta)
accuracy_boruta = accuracy_score(y_test_boruta, predictions_boruta)
confusion_matrix_boruta = confusion_matrix(y_test_boruta, predictions_boruta)

# Visualizza i risultati
print("Accuratezza RandomForest Boruta:", accuracy_boruta)
print("Matrice di confusione RandomForest Boruta:\n", confusion_matrix_boruta)

class_report_b_rf = classification_report(y_test_boruta, predictions_boruta)
print("Classification Report - Random Forest RFC:\n", class_report_b_rf)

# Estrai le etichette delle classi dal modello
class_labels_b_rf = model_boruta.classes_

# Per ogni classe, estrai FN e FP dalla matrice di confusione
for i, label in enumerate(class_labels_b_rf):
    fn = confusion_matrix_boruta[i, :].sum() - confusion_matrix_boruta[i, i]  # Somma della riga, escludendo la diagonale
    fp = confusion_matrix_boruta[:, i].sum() - confusion_matrix_boruta[i, i]  # Somma della colonna, escludendo la diagonale

    print(f"\nClasse: {label}")
    print(f"False Negative (FN): {fn}")
    print(f"False Positive (FP): {fp}")

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

# Separazione delle feature e del label
X_svm_b = new_dataset_boruta.drop(' Label', axis=1)  # Rimuovi la colonna del label
y_svm_b = new_dataset_boruta[' Label']

# Creazione dei set di allenamento e test
X_train_svm_b, X_test_svm_b, y_train_svm_b, y_test_svm_b = train_test_split(X_svm_b, y_svm_b, test_size=0.2, random_state=42)

# Allenamento del modello SVM
model_svm_b = SVC()
model_svm_b.fit(X_train_svm_b, y_train_svm_b)

# Valutazione del modello SVM
predictions_svm_b = model_svm_b.predict(X_test_svm_b)
accuracy_svm_b = accuracy_score(y_test_svm_b, predictions_svm_b)
confusion_matrix_svm_b = confusion_matrix(y_test_svm_b, predictions_svm_b)

# Visualizza i risultati
print("Accuratezza SVM RFC:", accuracy_svm_b)
print("Matrice di Confusione SVM RFC:\n", confusion_matrix_svm_b)

class_report_svm_b = classification_report(y_test_svm_b, predictions_svm_b)
print("Classification Report - Random Forest RFC:\n", class_report_svm_b)

# Estrai le etichette delle classi dal modello
class_labels_svm_b = model_svm_b.classes_

# Per ogni classe, estrai FN e FP dalla matrice di confusione
for i, label in enumerate(class_labels_b_rf):
    fn = confusion_matrix_svm_b[i, :].sum() - confusion_matrix_svm_b[i, i]  # Somma della riga, escludendo la diagonale
    fp = confusion_matrix_svm_b[:, i].sum() - confusion_matrix_svm_b[i, i]  # Somma della colonna, escludendo la diagonale

    print(f"\nClasse: {label}")
    print(f"False Negative (FN): {fn}")
    print(f"False Positive (FP): {fp}")

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

# Separazione delle feature e del label
X_svm_b = new_dataset_boruta.drop(' Label', axis=1)  # Rimuovi la colonna del label
y_svm_b = new_dataset_boruta[' Label']

# Creazione dei set di allenamento e test
X_train_svm_b, X_test_svm_b, y_train_svm_b, y_test_svm_b = train_test_split(X_svm_b, y_svm_b, test_size=0.2, random_state=42)

# Allenamento del modello SVM
model_svm_b = SVC()
model_svm_b.fit(X_train_svm_b, y_train_svm_b)

# Valutazione del modello SVM
predictions_svm_b = model_svm_b.predict(X_test_svm_b)
accuracy_svm_b = accuracy_score(y_test_svm_b, predictions_svm_b)
confusion_matrix_svm_b = confusion_matrix(y_test_svm_b, predictions_svm_b)

# Visualizza i risultati
print("Accuratezza SVM RFC:", accuracy_svm_b)
print("Matrice di Confusione SVM RFC:\n", confusion_matrix_svm_b)

In [ ]:
pca = PCA(n_components=50)
pca_res = pca.fit_transform(dataset_subset[feature_cols])

# Supponendo che i carichi delle variabili siano memorizzati nella variabile 'loadings'
loadings = np.abs(pca.components_[0])

# Ottieni gli indici delle variabili con i valori assoluti più alti
top_variables_indices = np.argsort(loadings)[::-1]

#weights_pca = {column_names[idx]: weight for weight, idx in enumerate(top_variables_indices, start=1)}

# Visualizza le prime N variabili più influenti
N = 50  # Puoi impostare questo valore in base a quanto vuoi visualizzare
top_variables = top_variables_indices[:N]

# Ottieni i nomi delle colonne corrispondenti agli indici delle variabili
column_names = dataset_subset.columns

# Prepara i dati per il grafico
top_variable_names = [column_names[idx] for idx in top_variables]
top_variable_loadings = [loadings[idx] for idx in top_variables]

weights_pca = {column_names[idx]: weight for weight, idx in enumerate(top_variables_indices, start=1)}

# Crea il grafico a barre
plt.figure(figsize=(12,8))
plt.bar(top_variable_names, top_variable_loadings, color='skyblue')
plt.xlabel('Carichi delle Variabili')
plt.title('Carichi delle Variabili più Influenti dopo PCA')
plt.xticks(rotation=45, ha='right')
plt.show()

total_variance = np.sum(np.var(dataset_subset, axis=0))
explained_variance = np.sum(pca.explained_variance_[:N])
accuracy_pca = (explained_variance/total_variance)*100

# Stampa i risultati
for idx in top_variables:
    variable_name = column_names[idx]
    print(f"Variabile: {variable_name}, Carico: {loadings[idx]}")

print(accuracy_pca)

data_subset = None

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix

# Assume che 'target' sia la colonna target del tuo dataset
target = dataset_subset[' Label']
X_train_pca, X_test_pca, y_train_pca, y_test_pca = train_test_split(pca_res[:, :20], target, test_size=0.2, random_state=42)

# Definizione del modello RandomForest
model_pca_rf = RandomForestClassifier()

# Definizione della griglia dei parametri da cercare
param_grid = {"max_depth": [3, None], "min_samples_split": [5, 10]}

# Ricerca dei parametri migliori con Halving Grid Search CV
search = HalvingGridSearchCV(model_pca_rf, param_grid, random_state=42).fit(X_train_pca, y_train_pca)
best_pca_params = search.best_params_

# Visualizzazione dei parametri migliori
print("Migliori parametri per RandomForestClassifier:", best_pca_params)

# Utilizzo dei migliori parametri per allenare il modello
best_model_pca = RandomForestClassifier(**best_pca_params)
best_model_pca.fit(X_train_pca, y_train_pca)

# Valutazione del modello RandomForest con i migliori parametri
predictions_pca_best = best_model_pca.predict(X_test_pca)
accuracy_pca_best = accuracy_score(y_test_pca, predictions_pca_best)
confusion_matrix_pca_best = confusion_matrix(y_test_pca, predictions_pca_best)

# Visualizza i risultati con i migliori parametri
print("Accuratezza RandomForest PCA con i migliori parametri:", accuracy_pca_best)
print("Matrice di confusione RandomForest PCA con i migliori parametri:\n", confusion_matrix_pca_best)

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

# Assume che 'target' sia la colonna target del tuo dataset
target = dataset_subset[' Label']
X_train_svm_pca, X_test_svm_pca, y_train_svm_pca, y_test_svm_pca = train_test_split(pca_res[:, :10], target, test_size=0.2, random_state=42)

# Support Vector Machine (SVM)
svm_model_pca = SVC()

# Definisci la griglia degli iperparametri
param_grid = {'C': [0.1, 0.5, 1, 2, 5, 10], 'kernel': ['linear', 'rbf', 'sigmoid', 'poly'], 'tol': [1e-3, 1e-2]}

# Applica Halving Grid Search CV
search_svm = HalvingGridSearchCV(svm_model_pca, param_grid, cv=5, factor=2, max_resources=110, min_resources=10, aggressive_elimination=False, random_state = 42)
search_svm.fit(X_train_svm_pca, y_train_svm_pca)

# Ottieni i migliori parametri
best_svm_params = search_svm.best_params_

# Visualizzazione dei parametri migliori
print("Migliori parametri per SVM:", best_svm_params)

# Utilizzo dei migliori parametri per allenare il modello
best_model_svm_pca = SVC(**best_svm_params)
best_model_svm_pca.fit(X_train_svm_pca, y_train_svm_pca)

# Valutazione del modello RandomForest con i migliori parametri
predictions_svm_pca = best_model_svm_pca.predict(X_test_svm_pca)
accuracy_svm_pca = accuracy_score(y_test_svm_pca, predictions_svm_pca)
confusion_matrix_svm_pca = confusion_matrix(y_test_svm_pca, predictions_svm_pca)

# Visualizza i risultati con i migliori parametri
print("Accuratezza SVM PCA con i migliori parametri:", accuracy_svm_pca)
print("Matrice di confusione SVM PCA con i migliori parametri:\n", confusion_matrix_svm_pca)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

# Assume che 'target' sia la colonna target del tuo dataset
target = dataset_subset[' Label']
X_train_pca, X_test_pca, y_train_pca, y_test_pca = train_test_split(pca_res[:, :30], target, test_size=0.2, random_state=42)

rf_model_pca = RandomForestClassifier()
rf_model_pca.fit(X_train_pca, y_train_pca)
rf_predictions_pca = rf_model_pca.predict(X_test_pca)

# Valutazione del modello RF
rf_accuracy_pca = accuracy_score(y_test_pca, rf_predictions_pca)
print("Accuracy Random Forest:", rf_accuracy_pca)

# Matrice di confusione RF
rf_conf_matrix_pca = confusion_matrix(y_test_pca, rf_predictions_pca)
print("\nMatrice di Confusione Random Forest:")
print(rf_conf_matrix_pca)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Assume che 'target' sia la colonna target del tuo dataset
target = dataset_subset[' Label']
X_train_pca, X_test_pca, y_train_pca, y_test_pca = train_test_split(pca_res[:, :20], target, test_size=0.2, random_state=42)

# Definizione del modello RandomForest
model_pca_rf = RandomForestClassifier()

# Definizione della griglia dei parametri da cercare
param_grid = {"max_depth": [3, None], "min_samples_split": [5, 10]}

# Ricerca dei parametri migliori con Halving Grid Search CV
search = HalvingGridSearchCV(model_pca_rf, param_grid, random_state=42).fit(X_train_pca, y_train_pca)
best_pca_params = search.best_params_

# Visualizzazione dei parametri migliori
print("Migliori parametri per RandomForestClassifier:", best_pca_params)

# Utilizzo dei migliori parametri per allenare il modello
best_model_pca = RandomForestClassifier(**best_pca_params)
best_model_pca.fit(X_train_pca, y_train_pca)

# Valutazione del modello RandomForest con i migliori parametri
predictions_pca_best = best_model_pca.predict(X_test_pca)
accuracy_pca_best = accuracy_score(y_test_pca, predictions_pca_best)
confusion_matrix_pca_best = confusion_matrix(y_test_pca, predictions_pca_best)

# Visualizza i risultati con i migliori parametri
print("Accuratezza RandomForest PCA con i migliori parametri:", accuracy_pca_best)
print("Matrice di confusione RandomForest PCA con i migliori parametri:\n", confusion_matrix_pca_best)

class_report_pca = classification_report(y_test_pca, predictions_pca_best)
print("Classification Report - Random Forest RFC:\n", class_report_pca)

# Estrai le etichette delle classi dal modello
class_labels_pca = best_model_pca.classes_

# Per ogni classe, estrai FN e FP dalla matrice di confusione
for i, label in enumerate(class_labels_pca):
    fn = confusion_matrix_pca_best[i, :].sum() - confusion_matrix_pca_best[i, i]  # Somma della riga, escludendo la diagonale
    fp = confusion_matrix_pca_best[:, i].sum() - confusion_matrix_pca_best[i, i]  # Somma della colonna, escludendo la diagonale

    print(f"\nClasse: {label}")
    print(f"False Negative (FN): {fn}")
    print(f"False Positive (FP): {fp}")

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

# Assume che 'target' sia la colonna target del tuo dataset
target = dataset_subset[' Label']
X_train_svm_pca, X_test_svm_pca, y_train_svm_pca, y_test_svm_pca = train_test_split(pca_res[:, :10], target, test_size=0.2, random_state=42)

# Support Vector Machine (SVM)
svm_model_pca = SVC()

# Definisci la griglia degli iperparametri
param_grid = {'C': [0.1, 0.5, 1, 2, 5, 10], 'kernel': ['linear', 'rbf', 'sigmoid', 'poly'], 'tol': [1e-3, 1e-2]}

# Applica Halving Grid Search CV
search_svm = HalvingGridSearchCV(svm_model_pca, param_grid, cv=5, factor=2, max_resources=110, min_resources=10, aggressive_elimination=False, random_state = 42)
search_svm.fit(X_train_svm_pca, y_train_svm_pca)

# Ottieni i migliori parametri
best_svm_params = search_svm.best_params_

# Visualizzazione dei parametri migliori
print("Migliori parametri per SVM:", best_svm_params)

# Utilizzo dei migliori parametri per allenare il modello
best_model_svm_pca = SVC(**best_svm_params)
best_model_svm_pca.fit(X_train_svm_pca, y_train_svm_pca)

# Valutazione del modello RandomForest con i migliori parametri
predictions_svm_pca = best_model_svm_pca.predict(X_test_svm_pca)
accuracy_svm_pca = accuracy_score(y_test_svm_pca, predictions_svm_pca)
confusion_matrix_svm_pca = confusion_matrix(y_test_svm_pca, predictions_svm_pca)

# Visualizza i risultati con i migliori parametri
print("Accuratezza SVM PCA con i migliori parametri:", accuracy_svm_pca)
print("Matrice di confusione SVM PCA con i migliori parametri:\n", confusion_matrix_svm_pca)

class_report_svm_pca = classification_report(y_test_svm_pca, predictions_svm_pca)
print("Classification Report - Random Forest RFC:\n", class_report_svm_pca)

# Estrai le etichette delle classi dal modello
class_labels_svm_pca = best_model_svm_pca.classes_

# Per ogni classe, estrai FN e FP dalla matrice di confusione
for i, label in enumerate(class_labels_svm_pca):
    fn = confusion_matrix_svm_pca[i, :].sum() - confusion_matrix_svm_pca[i, i]  # Somma della riga, escludendo la diagonale
    fp = confusion_matrix_svm_pca[:, i].sum() - confusion_matrix_svm_pca[i, i]  # Somma della colonna, escludendo la diagonale

    print(f"\nClasse: {label}")
    print(f"False Negative (FN): {fn}")
    print(f"False Positive (FP): {fp}")

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix

# Assume che 'target' sia la colonna target del tuo dataset
target = dataset_subset[' Label']
X_train_svm_pca, X_test_svm_pca, y_train_svm_pca, y_test_svm_pca = train_test_split(pca_res[:, :30], target, test_size=0.2, random_state=42)

# Support Vector Machine (SVM)
svm_model_pca = SVC()
svm_model_pca.fit(X_train_svm_pca, y_train_svm_pca)
svm_predictions_pca = svm_model_pca.predict(X_test_svm_pca)

# Valutazione del modello SVM
svm_accuracy_pca = accuracy_score(y_test_svm_pca, svm_predictions_pca)
print("Accuratezza SVM PCA:", svm_accuracy_pca)

# Matrice di confusione SVM
svm_conf_matrix_pca = confusion_matrix(y_test_svm_pca, svm_predictions_pca)
print("\nMatrice di Confusione SVM PCA:")
print(svm_conf_matrix_pca)

In [ ]:
# Supponendo che tu abbia già i pesi per i tre metodi
# Ad esempio, i pesi assegnati dalle tecniche di Borda e wBorda sono rispettivamente 'borda_weights_pca', 'borda_weights_boruta', 'borda_weights_rf'
# e 'wborda_weights_pca', 'wborda_weights_boruta', 'wborda_weights_rf'

total_accuracy = accuracy_pca + accuracy_boruta + accuracy_rf
alpha_pca = accuracy_pca / total_accuracy
alpha_boruta = accuracy_boruta / total_accuracy
alpha_rf = accuracy_rf / total_accuracy

# Calcola il ranking combinato
combined_ranking = {}

for feature in feature_df['Feature']:
    borda_score_pca = weights_pca.get(feature, 0)  # Se la feature non è presente, assegna 0
    borda_score_boruta = weights_boruta.get(feature, 0)  # Se la feature non è presente, assegna 0
    borda_score_rf = weights_rf.get(feature, 0)  # Se la feature non è presente, assegna 0

    wborda_score_pca = weights_pca.get(feature, 0)  # Se la feature non è presente, assegna 0
    wborda_score_boruta = weights_boruta.get(feature, 0)  # Se la feature non è presente, assegna 0
    wborda_score_rf = weights_rf.get(feature, 0)  # Se la feature non è presente, assegna 0

    # Calcola il ranking combinato
    combined_score = (
        alpha_pca * borda_score_pca + alpha_boruta * borda_score_boruta + alpha_rf * borda_score_rf +
        alpha_pca * wborda_score_pca + alpha_boruta * wborda_score_boruta + alpha_rf * wborda_score_rf
    )

    combined_ranking[feature] = combined_score

# Ordina le feature in base al ranking combinato
combined_ranking_sorted = {k: v for k, v in sorted(combined_ranking.items(), key=lambda item: item[1], reverse=True)}

# Stampa il ranking combinato
for feature, score in combined_ranking_sorted.items():
    print(f"Feature: {feature}, Punteggio Combinato: {score:.4f}")


In [ ]:
# Seleziona le prime 20 feature dal ranking combinato
top_features_combined = list(combined_ranking_sorted.keys())[:20]

# Aggiungi il label al set di feature selezionate
selected_features_combined = [' Label'] + top_features_combined

# Crea un nuovo dataset con solo le prime 20 feature e il label
new_dataset_combined = dataset_subset[selected_features_combined]

# Visualizza le prime righe del nuovo dataset
print(new_dataset_combined.head())

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix

# Separazione delle feature e del label nel nuovo dataset
X_rf_combined = new_dataset_combined.drop(' Label', axis=1)  # Rimuovi la colonna del label
y_rf_combined = new_dataset_combined[' Label']

# Creazione dei set di allenamento e test
X_train_rf_combined, X_test_rf_combined, y_train_rf_combined, y_test_rf_combined = train_test_split(X_rf_combined, y_rf_combined, test_size=0.2, random_state=42)

# Allenamento del modello RandomForest
model_rf_combined = RandomForestClassifier()

# Definizione della griglia dei parametri da cercare
param_grid = {"max_depth": [3, None], "min_samples_split": [5, 10]}

# Ricerca dei parametri migliori con Halving Grid Search CV
search = HalvingGridSearchCV(model_rf_combined, param_grid, random_state=42).fit(X_train_rf_combined, y_train_rf_combined)
best_combined_params = search.best_params_

# Visualizzazione dei parametri migliori
print("Migliori parametri per RandomForest:", best_combined_params)

# Utilizzo dei migliori parametri per allenare il modello
best_model_combined = RandomForestClassifier(**best_combined_params)
best_model_combined.fit(X_train_rf_combined, y_train_rf_combined)

# Valutazione del modello RandomForest con i migliori parametri
predictions_combined_best = best_model_combined.predict(X_test_rf_combined)
accuracy_combined_best = accuracy_score(y_test_rf_combined, predictions_combined_best)
confusion_matrix_combined_best = confusion_matrix(y_test_rf_combined, predictions_combined_best)

# Visualizza i risultati con i migliori parametri
print("Accuratezza RandomForest R.combinato con i migliori parametri:", accuracy_combined_best)
print("Matrice di confusione RandomForest R.combinato con i migliori parametri:\n", confusion_matrix_combined_best)

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

# Separazione delle feature e del label nel nuovo dataset
X_svm_combined = new_dataset_combined.drop(' Label', axis=1)  # Rimuovi la colonna del label
y_svm_combined = new_dataset_combined[' Label']

# Creazione dei set di allenamento e test
X_train_svm_combined, X_test_svm_combined, y_train_svm_combined, y_test_svm_combined = train_test_split(X_svm_combined, y_svm_combined, test_size=0.2, random_state=42)

# Allenamento del modello SVM
model_svm_combined = SVC()

# Definisci la griglia degli iperparametri
param_grid = {'C': [0.1, 0.5, 1, 2, 5, 10], 'kernel': ['linear', 'rbf', 'sigmoid', 'poly'], 'tol': [1e-3, 1e-2]}

# Applica Halving Grid Search CV
search_svm = HalvingGridSearchCV(model_svm_combined, param_grid, cv=5, factor=2, max_resources=110, min_resources=10, aggressive_elimination=False, random_state = 42)
search_svm.fit(X_train_svm_combined, y_train_svm_combined)

# Ottieni i migliori parametri
best_svm_params = search_svm.best_params_

# Visualizzazione dei parametri migliori
print("Migliori parametri per SVM:", best_svm_params)

# Utilizzo dei migliori parametri per allenare il modello
best_model_svm_combined = SVC(**best_svm_params)
best_model_svm_combined.fit(X_train_svm_combined, y_train_svm_combined)

# Valutazione del modello RandomForest con i migliori parametri
predictions_svm_combined = best_model_svm_combined.predict(X_test_svm_combined)
accuracy_svm_combined = accuracy_score(y_test_svm_combined, predictions_svm_combined)
confusion_matrix_svm_combined = confusion_matrix(y_test_svm_combined, predictions_svm_combined)

# Visualizza i risultati con i migliori parametri
print("Accuratezza SVM R.combinato con i migliori parametri:", accuracy_svm_combined)
print("Matrice di confusione SVM R.combinato con i migliori parametri:\n", confusion_matrix_svm_combined)

In [ ]:
#RF per ranking combinato
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

# Separazione delle feature e del label nel nuovo dataset
X_rf_combined = new_dataset_combined.drop(' Label', axis=1)  # Rimuovi la colonna del label
y_rf_combined = new_dataset_combined[' Label']

# Creazione dei set di allenamento e test
X_train_rf_combined, X_test_rf_combined, y_train_rf_combined, y_test_rf_combined = train_test_split(X_rf_combined, y_rf_combined, test_size=0.2, random_state=42)

# Allenamento del modello RandomForest
model_rf_combined = RandomForestClassifier()
model_rf_combined.fit(X_train_rf_combined, y_train_rf_combined)

# Valutazione del modello RandomForest
predictions_rf_combined = model_rf_combined.predict(X_test_rf_combined)
accuracy_rf_combined = accuracy_score(y_test_rf_combined, predictions_rf_combined)
confusion_matrix_rf_combined = confusion_matrix(y_test_rf_combined, predictions_rf_combined)

# Visualizza i risultati per RandomForest
print("Accuratezza RandomForest Ranking Combinato:", accuracy_rf_combined)
print("Matrice Confusione RandomForest Ranking Combinato:\n", confusion_matrix_rf_combined)


In [ ]:
#RF per ranking combinato
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Separazione delle feature e del label nel nuovo dataset
X_rf_combined = new_dataset_combined.drop(' Label', axis=1)  # Rimuovi la colonna del label
y_rf_combined = new_dataset_combined[' Label']

# Creazione dei set di allenamento e test
X_train_rf_combined, X_test_rf_combined, y_train_rf_combined, y_test_rf_combined = train_test_split(X_rf_combined, y_rf_combined, test_size=0.2, random_state=42)

# Allenamento del modello RandomForest
model_rf_combined = RandomForestClassifier()
model_rf_combined.fit(X_train_rf_combined, y_train_rf_combined)

# Valutazione del modello RandomForest
predictions_rf_combined = model_rf_combined.predict(X_test_rf_combined)
accuracy_rf_combined = accuracy_score(y_test_rf_combined, predictions_rf_combined)
confusion_matrix_rf_combined = confusion_matrix(y_test_rf_combined, predictions_rf_combined)

# Visualizza i risultati per RandomForest
print("Accuratezza RandomForest Ranking Combinato:", accuracy_rf_combined)
print("Matrice Confusione RandomForest Ranking Combinato:\n", confusion_matrix_rf_combined)

class_report_combined = classification_report(y_test_rf_combined, predictions_rf_combined)
print("Classification Report - Random Forest RFC:\n", class_report_combined)

# Estrai le etichette delle classi dal modello
class_labels_combined = model_rf_combined.classes_

# Per ogni classe, estrai FN e FP dalla matrice di confusione
for i, label in enumerate(class_labels_combined):
    fn = confusion_matrix_rf_combined[i, :].sum() - confusion_matrix_rf_combined[i, i]  # Somma della riga, escludendo la diagonale
    fp = confusion_matrix_rf_combined[:, i].sum() - confusion_matrix_rf_combined[i, i]  # Somma della colonna, escludendo la diagonale

    print(f"\nClasse: {label}")
    print(f"False Negative (FN): {fn}")
    print(f"False Positive (FP): {fp}")

In [ ]:
#SVM per ranking combinato
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

# Separazione delle feature e del label nel nuovo dataset
X_svm_combined = new_dataset_combined.drop(' Label', axis=1)  # Rimuovi la colonna del label
y_svm_combined = new_dataset_combined[' Label']

# Creazione dei set di allenamento e test
X_train_svm_combined, X_test_svm_combined, y_train_svm_combined, y_test_svm_combined = train_test_split(X_svm_combined, y_svm_combined, test_size=0.2, random_state=42)

# Allenamento del modello SVM
model_svm_combined = SVC()
model_svm_combined.fit(X_train_svm_combined, y_train_svm_combined)

# Valutazione del modello SVM
predictions_svm_combined = model_svm_combined.predict(X_test_svm_combined)
accuracy_svm_combined = accuracy_score(y_test_svm_combined, predictions_svm_combined)
confusion_matrix_svm_combined = confusion_matrix(y_test_svm_combined, predictions_svm_combined)

# Visualizza i risultati per SVM
print("Accuracy SVM (con le prime 20 feature):", accuracy_svm_combined)
print("Confusion Matrix SVM (con le prime 20 feature):\n", confusion_matrix_svm_combined)

class_report_svm_combined = classification_report(y_test_svm_combined, predictions_svm_combined)
print("Classification Report - Random Forest RFC:\n", class_report_svm_combined)

# Estrai le etichette delle classi dal modello
class_labels_svm_combined = model_svm_combined.classes_

# Per ogni classe, estrai FN e FP dalla matrice di confusione
for i, label in enumerate(class_labels_svm_combined):
    fn = confusion_matrix_svm_combined[i, :].sum() - confusion_matrix_svm_combined[i, i]  # Somma della riga, escludendo la diagonale
    fp = confusion_matrix_svm_combined[:, i].sum() - confusion_matrix_svm_combined[i, i]  # Somma della colonna, escludendo la diagonale

    print(f"\nClasse: {label}")
    print(f"False Negative (FN): {fn}")
    print(f"False Positive (FP): {fp}")

In [ ]:
#SVM per ranking combinato
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

# Separazione delle feature e del label nel nuovo dataset
X_svm_combined = new_dataset_combined.drop(' Label', axis=1)  # Rimuovi la colonna del label
y_svm_combined = new_dataset_combined[' Label']

# Creazione dei set di allenamento e test
X_train_svm_combined, X_test_svm_combined, y_train_svm_combined, y_test_svm_combined = train_test_split(X_svm_combined, y_svm_combined, test_size=0.2, random_state=42)

# Allenamento del modello SVM
model_svm_combined = SVC()
model_svm_combined.fit(X_train_svm_combined, y_train_svm_combined)

# Valutazione del modello SVM
predictions_svm_combined = model_svm_combined.predict(X_test_svm_combined)
accuracy_svm_combined = accuracy_score(y_test_svm_combined, predictions_svm_combined)
confusion_matrix_svm_combined = confusion_matrix(y_test_svm_combined, predictions_svm_combined)

# Visualizza i risultati per SVM
print("Accuracy SVM (con le prime 20 feature):", accuracy_svm_combined)
print("Confusion Matrix SVM (con le prime 20 feature):\n", confusion_matrix_svm_combined)


In [ ]:
tsne = TSNE(n_components=2, verbose=0, perplexity=100, n_iter=1000)
tsne_res = tsne.fit_transform(pca_res)
print("done")

In [ ]:
dataset_subset['tsne_firstD'] = tsne_res[:,0]
dataset_subset['tsne_secondD'] = tsne_res[:,1]
dataset_subset

In [ ]:
custom_palette = [
    (0, 0, 0),       # Nero
    (1, 1, 0),       # Giallo
    (1, 0, 0),       # Rosso
    (0, 1, 0),       # Verde
    (1, 0.549, 0),   # Arancione
    (1, 0, 1),       # Fuchsia
    (0, 1, 1),       # Ciano
    (0, 0, 1),       # Blu
    (0.502, 0, 0.502),  # Lilla
    (0.502, 0.502, 0),  # Olive
    (0, 0.749, 1)   # Turchese
]

In [ ]:
plt.figure(figsize=(16,16))

sns.scatterplot(
    x="tsne_firstD",
    y="tsne_secondD",
    palette=custom_palette,
    data=dataset_subset,
    hue=' Label',
    legend="full",
    alpha=0.75
)

plt.xlabel('tsne_firstD')
plt.ylabel('tsne_secondD')
plt.title('Anomaly Detection')
plt.show()